In [48]:
# Uncomment and run this cell if the required libraries are not installed
# !pip install dash pandas plotly

from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px


In [49]:
# Load the dataset
df = pd.read_csv("C:/Users/User/OneDrive - University of Exeter/PostGradute/Analytics and Visualization/CW2/cleaned_hr_data.csv")

# Decoding mappings
department_mapping = {0: "Sales", 1: "Research & Development", 2: "Human Resources"}
job_role_mapping = {
    0: "Manager", 1: "Sales Executive", 2: "Human Resources", 3: "Manufacturing Director",
    4: "Research Director", 5: "Research Scientist", 6: "Sales Representative",
    7: "Laboratory Technician", 8: "Healthcare Representative"
}
business_travel_mapping = {0: "Non-Travel", 1: "Travel_Rarely", 2: "Travel_Frequently"}

# Apply mappings to dataset
df["Department"] = df["Department"].map(department_mapping)
df["JobRole"] = df["JobRole"].map(job_role_mapping)
df["BusinessTravel"] = df["BusinessTravel"].map(business_travel_mapping)


In [50]:
# Total Employees, Attrition Numbers, and Attrition Rate
total_employees = df.shape[0]
total_attrition = df[df["Attrition"] == 1].shape[0]
attrition_rate = (total_attrition / total_employees) * 100

# Pie Chart Data (Stay vs Leave)
attrition_pie = df["Attrition"].value_counts(normalize=True) * 100

# Gender Attrition Rate
gender_attrition = df[df["Attrition"] == 1].groupby("Gender").size() / df.groupby("Gender").size() * 100

# Department and Job Role Attrition
department_attrition = df.groupby("Department")["Attrition"].mean() * 100
job_role_attrition = df[df["Attrition"] == 1].groupby(["Department", "JobRole"]).size() / df.groupby(["Department", "JobRole"]).size() * 100

# Age Group Attrition
age_bins = [18, 25, 35, 45, 60]
age_labels = ["18-25", "26-35", "36-45", "46-60"]
df["AgeGroup"] = pd.cut(df["Age"], bins=age_bins, labels=age_labels, right=False)
age_group_attrition = df[df["Attrition"] == 1].groupby("AgeGroup").size() / df.groupby("AgeGroup").size() * 100

# Years Since Last Promotion Attrition
promotion_bins = [0, 2, 4, 6, 8, df["YearsSinceLastPromotion"].max() + 1]
promotion_labels = ["0-2", "2-4", "4-6", "6-8", "8+"]
df["PromotionGroup"] = pd.cut(df["YearsSinceLastPromotion"], bins=promotion_bins, labels=promotion_labels, right=False)
promotion_attrition = df[df["Attrition"] == 1].groupby("PromotionGroup").size() / df.groupby("PromotionGroup").size() * 100

# Stock Level Attrition
stock_level_attrition = df[df["Attrition"] == 1].groupby("StockOptionLevel").size() / df.groupby("StockOptionLevel").size() * 100

# Business Travel Attrition
business_travel_attrition = df[df["Attrition"] == 1].groupby("BusinessTravel").size() / df.groupby("BusinessTravel").size() * 100


C:\Users\User\AppData\Local\Temp\ipykernel_31256\124539601.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_31256\124539601.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_31256\124539601.py:26: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_31256\124539601.py:26: FutureWarning:

The de

In [51]:

# Pie Chart: Stay vs Leave
fig_pie = px.pie(
    names=["Employee Stay", "Employee Leave"],
    values=attrition_pie.values,
    color=["Employee Stay", "Employee Leave"],
    color_discrete_map={"Employee Stay": "#006078", "Employee Leave": "#E37C78"},
    title="Employee Stay vs Leave"
)

# Gender Attrition (using Male and Female percentage)
fig_gender_attrition = px.bar(
    x=gender_attrition.index,
    y=gender_attrition.values,
    labels={"x": "Gender", "y": "Attrition Rate (%)"},
    title="Attrition by Gender",
    text=gender_attrition.values,
    color=gender_attrition.index,
    color_discrete_map={"Male": "#006078", "Female": "#E37C78"}
)
fig_gender_attrition.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Bar Chart: Attrition by Department
fig_department_bar = px.bar(
    x=department_attrition.index,
    y=department_attrition.values,
    labels={"x": "Department", "y": "Attrition Rate (%)"},
    title="Attrition by Department",
    text=department_attrition.values
)
fig_department_bar.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Job Role Attrition (by Department)
fig_job_role_attrition = px.bar(
    job_role_attrition.reset_index(),
    x="JobRole",
    y=0,
    color="Department",
    labels={"JobRole": "Job Role", 0: "Attrition Rate (%)", "Department": "Department"},
    title="Attrition by Job Role",
    text=0,
    barmode="group"
)
fig_job_role_attrition.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Horizontal Bar Chart: Attrition by Age Group
fig_age_group_bar = px.bar(
    y=age_group_attrition.index,
    x=age_group_attrition.values,
    orientation="h",
    labels={"x": "Attrition Rate (%)", "y": "Age Group"},
    title="Attrition by Age Group"
)

# Horizontal Bar Chart: Attrition by Years Since Last Promotion
fig_promotion_bar = px.bar(
    y=promotion_attrition.index,
    x=promotion_attrition.values,
    orientation="h",
    labels={"x": "Attrition Rate (%)", "y": "Years Since Last Promotion"},
    title="Attrition by Years Since Last Promotion"
)

# Horizontal Bar Chart: Attrition by Stock Level
fig_stock_level_bar = px.bar(
    y=stock_level_attrition.index,
    x=stock_level_attrition.values,
    orientation="h",
    labels={"x": "Attrition Rate (%)", "y": "Stock Option Level"},
    title="Attrition by Stock Option Level"
)

# Horizontal Bar Chart: Attrition by Business Travel
fig_business_travel_bar = px.bar(
    y=business_travel_attrition.index,
    x=business_travel_attrition.values,
    orientation="h",
    labels={"x": "Attrition Rate (%)", "y": "Business Travel"},
    title="Attrition by Business Travel"
)


In [52]:
# Initialize Dash app
app = Dash(__name__)


In [53]:
app.layout = html.Div(
    style={"padding": "20px", "backgroundColor": "#FAEFED", "fontFamily": "Arial, sans-serif"},
    children=[
        # Title Section
        html.H1("Human Resource Attrition Analytics", style={"textAlign": "center"}),

        # High-Level Text Boxes
       # Updated High-Level Text Boxes with Proper Styling
html.Div(
    style={
        "display": "flex",
        "justifyContent": "space-between",
        "alignItems": "center",
        "padding": "10px 50px",  # Adds spacing at the top and sides
        "marginTop": "20px",
    },
    children=[
        html.Div(
            children=[
                html.H5("Total Employees", style={"color": "white", "marginBottom": "0px", "fontSize": "18px"}),
                html.H3(f"{total_employees}", style={"color": "white", "marginTop": "5px", "fontSize": "28px"})
            ],
            style={
                "backgroundColor": "#1E6BA0",  # Updated Blue Color
                "padding": "20px",
                "borderRadius": "10px",
                "textAlign": "center",
                "boxShadow": "0 4px 6px rgba(0, 0, 0, 0.1)",
                "width": "30%"
            }
        ),
        html.Div(
            children=[
                html.H5("Total Employees Left", style={"color": "white", "marginBottom": "0px", "fontSize": "18px"}),
                html.H3(f"{total_attrition}", style={"color": "white", "marginTop": "5px", "fontSize": "28px"})
            ],
            style={
                "backgroundColor": "#F57C00",  # Updated Orange Color
                "padding": "20px",
                "borderRadius": "10px",
                "textAlign": "center",
                "boxShadow": "0 4px 6px rgba(0, 0, 0, 0.1)",
                "width": "30%"
            }
        ),
        html.Div(
            children=[
                html.H5("Overall Attrition Rate", style={"color": "white", "marginBottom": "0px", "fontSize": "18px"}),
                html.H3(f"{attrition_rate:.2f}%", style={"color": "white", "marginTop": "5px", "fontSize": "28px"})
            ],
            style={
                "backgroundColor": "#43A047",  # Updated Green Color
                "padding": "20px",
                "borderRadius": "10px",
                "textAlign": "center",
                "boxShadow": "0 4px 6px rgba(0, 0, 0, 0.1)",
                "width": "30%"
            }
        ),
    ]
),



        # Pie Chart: Stay vs Leave
        dcc.Graph(figure=fig_pie),

        # Bar Chart: Gender Attrition
        dcc.Graph(figure=fig_gender_attrition),
        # Gender Attrition Section with Icons and Text Boxes
html.Div(
    style={
        "display": "flex",
        "justifyContent": "center",
        "alignItems": "center",
        "marginTop": "30px",
    },
    children=[
        # Male Attrition
        html.Div(
            style={
                "textAlign": "center",
                "marginRight": "50px",
                "backgroundColor": "#1E6BA0",
                "borderRadius": "10px",
                "width": "200px",
                "padding": "15px",
                "boxShadow": "0 4px 6px rgba(0, 0, 0, 0.1)"
            },
            children=[
                html.Img(
                    src="C:/Users/User/OneDrive - University of Exeter/PostGradute/Analytics and Visualization/CW2/Visualization Project HR Management Data Analytic part_files/assets/Male_icon.png",  # Replace with your local or online path
                    style={"width": "50px", "height": "50px", "marginBottom": "10px"}
                ),
                html.H5("Male Attrition", style={"color": "white", "marginBottom": "5px"}),
                html.H3(f"{gender_attrition[0]:.2f}%", style={"color": "white", "marginTop": "0"})
            ]
        ),
        # Female Attrition
        html.Div(
            style={
                "textAlign": "center",
                "backgroundColor": "#1E6BA0",
                "borderRadius": "10px",
                "width": "200px",
                "padding": "15px",
                "boxShadow": "0 4px 6px rgba(0, 0, 0, 0.1)"
            },
            children=[
                html.Img(
                    src="C:/Users/User/OneDrive - University of Exeter/PostGradute/Analytics and Visualization/CW2/Visualization Project HR Management Data Analytic part_files/assets/Female_icon.png",  # Replace with your local or online path
                    style={"width": "50px", "height": "50px", "marginBottom": "10px"}
                ),
                html.H5("Female Attrition", style={"color": "white", "marginBottom": "5px"}),
                html.H3(f"{gender_attrition[1]:.2f}%", style={"color": "white", "marginTop": "0"})
            ]
        ),
    ]
),


        # Vertical Bar Chart: Attrition by Department
        dcc.Graph(figure=fig_department_bar),

        # Horizontal Bar Chart: Age Group
        dcc.Graph(figure=fig_age_group_bar),

        # Horizontal Bar Chart: Years Since Last Promotion
        dcc.Graph(figure=fig_promotion_bar),

        # Horizontal Bar Chart: Stock Option Level
        dcc.Graph(figure=fig_stock_level_bar),

        # Horizontal Bar Chart: Business Travel
        dcc.Graph(figure=fig_business_travel_bar),

        # Job Role Attrition
        dcc.Graph(figure=fig_job_role_attrition),
    ]
)


In [54]:
# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8080)
